In [6]:
import numpy as np
import torch
from tqdm import tqdm

In [7]:
# 중앙을 중심으로 지킬앤 하이드 처럼 좌우에 컷믹스
def rand_bbox_normal_left_right(size, lam):
    H = size[2]
    W = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)


    cx = np.random.randn() + W//2
    cy = np.random.randn() + H//2

    # 패치의 4점
    bbx1 = np.clip(cx - cut_w // 2, 0, W//2)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W//2)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return int(bbx1), int(bby1), int(bbx2), int(bby2)

In [8]:
for data, target in tqdm(self.data_loader, leave=False):
            
    data = data.to(self.device)
    target = target.to(self.device)

    # 일정 확률로 컷믹스
    if np.random.random() > 0.5:

        rand_index = torch.randperm(data.size()[0]) # 패치에 사용할 label
        target_a = target # 원본 이미지 label
        target_b = target[rand_index] # 패치 이미지 label  
        lam = np.random.beta(1.0, 1.0)     
        bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)

        # 원본 데이터에 컷믹스 패치
        data[:, :, bbx1:bbx2, bby1:bby2] = data[rand_index, :, bbx1:bbx2, bby1:bby2]

        # 원본 이미지와 패치 이미지의 넓이 비율
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))

        # 예측은 레이블 1개
        pred = self.model(data)

        # 원본 이미지의 레이블과 패치 이미지의 레이블에 대해 loss 가중합
        loss = self.criterion(pred, target_a) * lam + self.criterion(pred, target_b) * (1. - lam)
    
    # 컷믹스 안하면 그냥 별거 없음
    else:
        pred = self.model(data)
        loss = self.criterion(pred, target)
        
        
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

NameError: name 'tqdm' is not defined